## SEC FILINGS
### JOE-BUYDEM-DIPS

v5 - 5/18/2022

SEE https://github.com/nss-data-science-cohort-5/sec_filings_for_real-joebuydemdips

In [1]:
# LIBRARY IMPORTS

from bs4 import BeautifulSoup as BS
import glob
# from IPython.core.display import display, HTML
from nltk import sent_tokenize, word_tokenize, \
    regexp_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import numpy as np
import pandas as pd
# import re
import spacy
# from spacy import displacy   # displacy will eat all your RAM
from spacy.lang.en import English
from spacy.matcher import Matcher#, PhraseMatcher
# from spacy.symbols import nsubj, VERB
from spacy.tokens import Doc, Span

### FIRST GET TEXT FOR ALL 10k FILES

In [2]:
filepaths = glob.glob('../data/*.htm*')

In [22]:
# VAMSI'S CODE FROM 5/6/2022

ten_ks = {}

for filepath in filepaths:
    with open(filepath, 'r') as f:
        content = f.read()
        soup = BS(content, 
                  'html5lib')

        # Kill all script and style elements
        for script in soup(['script', 
                            'style']):
            script.extract()

        # Get text    
        text = soup.get_text()

        # Break into lines and remove leading and trailing space on each
        lines = (line.strip() \
                 for line in text.splitlines())

        # Break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines \
                  for phrase in line.split("  "))

        # Drop blank lines
        ht_text = '\n'.join(chunk for chunk in chunks if chunk)

        # Append to dictionary
        ten_ks[filepath] = ht_text

In [ ]:
# ten_ks[filepath][0:1000]

# amrk = ten_ks['../data/amrk-10k_20210630.htm']

In [23]:
ht_text[0:1000]

'siri-2021123100009089372021FYfalseP3MP3Yhttp://fasb.org/us-gaap/2021-01-31#PropertyPlantAndEquipmentAndFinanceLeaseRightOfUseAssetAfterAccumulatedDepreciationAndAmortizationhttp://fasb.org/us-gaap/2021-01-31#PropertyPlantAndEquipmentAndFinanceLeaseRightOfUseAssetAfterAccumulatedDepreciationAndAmortizationhttp://fasb.org/us-gaap/2021-01-31#LongTermDebtAndCapitalLeaseObligationsCurrenthttp://fasb.org/us-gaap/2021-01-31#LongTermDebtAndCapitalLeaseObligationsCurrenthttp://fasb.org/us-gaap/2021-01-31#LongTermDebtAndCapitalLeaseObligationshttp://fasb.org/us-gaap/2021-01-31#LongTermDebtAndCapitalLeaseObligations0.1537797P3YP1Y00009089372021-01-012021-12-3100009089372021-06-30iso4217:USD00009089372022-01-28xbrli:shares0000908937us-gaap:SubscriptionAndCirculationMember2021-01-012021-12-310000908937us-gaap:SubscriptionAndCirculationMember2020-01-012020-12-310000908937us-gaap:SubscriptionAndCirculationMember2019-01-012019-12-310000908937us-gaap:AdvertisingMember2021-01-012021-12-310000908937us-g

In [24]:
type(ht_text)

str

In [15]:
type(ten_ks)

list

In [ ]:
# amrk[0:1000]

In [17]:
ten_k_str = ten_ks.to_string()

AttributeError: 'list' object has no attribute 'to_string'

#### TOKENIZING

In [25]:
sent_doc = [sentence.lower() for sentence \
            in sent_tokenize(ht_text)]

In [26]:
sent_doc[0:1000]

['siri-2021123100009089372021fyfalsep3mp3yhttp://fasb.org/us-gaap/2021-01-31#propertyplantandequipmentandfinanceleaserightofuseassetafteraccumulateddepreciationandamortizationhttp://fasb.org/us-gaap/2021-01-31#propertyplantandequipmentandfinanceleaserightofuseassetafteraccumulateddepreciationandamortizationhttp://fasb.org/us-gaap/2021-01-31#longtermdebtandcapitalleaseobligationscurrenthttp://fasb.org/us-gaap/2021-01-31#longtermdebtandcapitalleaseobligationscurrenthttp://fasb.org/us-gaap/2021-01-31#longtermdebtandcapitalleaseobligationshttp://fasb.org/us-gaap/2021-01-31#longtermdebtandcapitalleaseobligations0.1537797p3yp1y00009089372021-01-012021-12-3100009089372021-06-30iso4217:usd00009089372022-01-28xbrli:shares0000908937us-gaap:subscriptionandcirculationmember2021-01-012021-12-310000908937us-gaap:subscriptionandcirculationmember2020-01-012020-12-310000908937us-gaap:subscriptionandcirculationmember2019-01-012019-12-310000908937us-gaap:advertisingmember2021-01-012021-12-310000908937us-

In [27]:
sentences = []
stemmer = PorterStemmer()
stock_stem = stemmer.stem('stock')
shares_stem = stemmer.stem('shares')

for idx, sentence in enumerate(sent_doc):
    if 'repurchase program' in sentence:
        chunk = sent_doc[idx-5:idx+6]
        for x in chunk:
            if stock_stem in x or shares_stem in x:
                sentences.append(chunk)
                break

In [28]:
sentences

[['in general, these leases and licenses are for space on building rooftops and communications towers.',
  'none of these individual locations are material to our business or operations.pandoralocationpurposeown/leaseoakland, caoffice and technical/engineering facilitiesleasenew york, nyoffice, sales and studio/production facilitiesleaseatlanta, gaoffice, sales and technical/engineering facilitiesleasesanta monica, caoffice and sales facilitiesleasewe also lease other small facilities that we use as offices for our sales and office personnel.',
  'these facilities are not material to our business or operations.item 3.',
  'legal proceedingsfor a discussion of our “legal proceedings,” refer to note 16 in the notes to our audited consolidated financial statements in this annual report on form 10-k.item 4.',
  'mine safety disclosuresnot applicable.30table of contentspart iiitem 5.',
  "market for registrant's common equity, related stockholder matters and issuer purchases of equity secur

In [29]:
nlp = spacy.load('en_core_web_sm')
values = []
for chunk in sentences:
    for sentence in chunk:
        text = nlp(sentence)
        for token in text:
            if token.like_num:
                #values.append(text[token.i-10:token.i+11])
                values.append(sentence)
                break
values

['these facilities are not material to our business or operations.item 3.',
 'legal proceedingsfor a discussion of our “legal proceedings,” refer to note 16 in the notes to our audited consolidated financial statements in this annual report on form 10-k.item 4.',
 'mine safety disclosuresnot applicable.30table of contentspart iiitem 5.',
 "market for registrant's common equity, related stockholder matters and issuer purchases of equity securitiesour common stock is traded on the nasdaq global select market under the symbol “siri.”\non january\xa028, 2022, there were approximately 6,529 record holders of our common stock.issuer purchases of equity securitiesas of december\xa031, 2021, our board of directors had authorized us to repurchase an aggregate of $18.0 billion of our common stock and have not establish an end date for this stock repurchase program.",
 'shares of common stock may be purchased from time to time on the open market, pursuant to pre-set trading plans meeting the requ

#### MATCHING

In [30]:
matcher = Matcher(nlp.vocab)

In [31]:
repur_1 = [{'LEMMA': 'repurchase'}]

In [32]:
matcher.add('REPURCHASE_PATTERN', [repur_1])

In [33]:
repur_2 = [{'LEMMA': 'authoriz'}]

In [34]:
matcher.add('AUTH_PATTERN', [repur_2])

In [35]:
repur_3 = [{'LEMMA': 'share'}]

In [36]:
matcher.add('SHARE_PATTERN', [repur_3])

In [37]:
val_doc = []

for value in values:
    val_doc.append(nlp(value))

In [38]:
val_doc[0:1000]

[these facilities are not material to our business or operations.item 3.,
 legal proceedingsfor a discussion of our “legal proceedings,” refer to note 16 in the notes to our audited consolidated financial statements in this annual report on form 10-k.item 4.,
 mine safety disclosuresnot applicable.30table of contentspart iiitem 5.,
 market for registrant's common equity, related stockholder matters and issuer purchases of equity securitiesour common stock is traded on the nasdaq global select market under the symbol “siri.”
 on january 28, 2022, there were approximately 6,529 record holders of our common stock.issuer purchases of equity securitiesas of december 31, 2021, our board of directors had authorized us to repurchase an aggregate of $18.0 billion of our common stock and have not establish an end date for this stock repurchase program.,
 shares of common stock may be purchased from time to time on the open market, pursuant to pre-set trading plans meeting the requirements of rul

In [43]:
ent_match_cont = []

for doc in val_doc:
    ent_match_list = []
    for ent in doc.ents:
        if ent.label_ == 'CARDINAL':
            ent_match_list.append((matcher(doc), ent.text, \
                                   ent.start_char, ent.end_char, \
                                   ent.label_))
        if ent.label_ == 'DATE':
            ent_match_list.append((matcher(doc), ent.text, \
                                   ent.start_char, ent.end_char, \
                                   ent.label_))
        if ent.label_ == 'MONEY':
            ent_match_list.append((matcher(doc), ent.text, \
                                   ent.start_char, ent.end_char, \
                                   ent.label_))  
        if ent.label_ == 'SHARES':
            ent_match_list.append((matcher(doc), ent.text, \
                                   ent.start_char, ent.end_char, \
                                   ent.label_))            
    ent_match_cont.append({'TEXT': doc.text, 
                           'ENTITIES': ent_match_list})

In [40]:
ent_match_cont[0:1000]

[{'TEXT': 'these facilities are not material to our business or operations.item 3.',
  'ENTITIES': [([], '3', 69, 70, 'CARDINAL')]},
 {'TEXT': 'legal proceedingsfor a discussion of our “legal proceedings,” refer to note 16 in the notes to our audited consolidated financial statements in this annual report on form 10-k.item 4.',
  'ENTITIES': [([], '16', 76, 78, 'CARDINAL')]},
 {'TEXT': 'mine safety disclosuresnot applicable.30table of contentspart iiitem 5.',
  'ENTITIES': [([], '5', 69, 70, 'CARDINAL')]},
 {'TEXT': "market for registrant's common equity, related stockholder matters and issuer purchases of equity securitiesour common stock is traded on the nasdaq global select market under the symbol “siri.”\non january\xa028, 2022, there were approximately 6,529 record holders of our common stock.issuer purchases of equity securitiesas of december\xa031, 2021, our board of directors had authorized us to repurchase an aggregate of $18.0 billion of our common stock and have not establis